In [1]:

from ib_async import *
from data_and_research import ac
import pandas as pd
import datetime, math
from arcticdb import Arctic, QueryBuilder, LibraryOptions
import yfinance as yf

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager import StrategyManager
    from strategy_manager.strategies.temple_w_fill import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfoliomanager import PortfolioManager
    from data_and_research import ac, initialize_db
    from broker.utils import FXCache, create_position_dict, create_trade_entry ,calculate_pnl, detect_duplicate_trade

except:
    from strategy_manager import StrategyManager
    from strategy_manager.strategies.temple_w_fill import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfoliomanager import PortfolioManager
    from data_and_research import ac, initialize_db
    from broker.utils import FXCache, create_position_dict, create_trade_entry ,calculate_pnl, detect_duplicate_trade



Connected to ArcticDB at ../db


In [2]:
strategy_symbol = "TAAPL"
trade = Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=499, clientId=1, permId=1987284107, action='BUY', totalQuantity=1.0, lmtPrice=0.0, auxPrice=0.0, orderRef='TAAPL', algoStrategy='Adaptive', algoParams=[TagValue(tag='adaptivePriority', value='Urgent')]), orderStatus=OrderStatus(orderId=499, status='Filled', filled=1.0, remaining=0.0, avgFillPrice=229.23, permId=1987284107, parentId=0, lastFillPrice=229.23, clientId=1, whyHeld='', mktCapPrice=0.0), fills=[Fill(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), execution=Execution(execId='0000dc8f.66ebbc62.01.01', time=datetime.datetime(2024, 9, 19, 19, 14, 49, tzinfo=datetime.timezone.utc), acctNumber='DU3180649', exchange='IBKRATS', side='BOT', shares=1.0, price=229.23, permId=1987284107, clientId=1, orderId=499, liquidation=0, cumQty=1.0, avgPrice=229.23, orderRef='TAAPL', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2, pendingPriceRevision=False), commissionReport=CommissionReport(execId='0000dc8f.66ebbc62.01.01', commission=1.000052, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc))], log=[TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 48, 706996, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Submitted', message='Fill 1.0@229.23', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Filled', message='', errorCode=0)], advancedError='')

In [4]:
ib = connect_to_IB(clientid=34)
self = PortfolioManager(ib)


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfarm.nj; eufarm; cashfarm; usfarm; euhmds; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfarm.nj; eufarm; cashfarm; usfarm; euhmds; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.


In [5]:
# hae a look at the data from portfolio lib
self.portfolio_library.read(self.account_id).data

,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
2024-09-19 22:15:36.758804000,AAPL,STK,1.0,0.093992,230.230052,228.690002,-0.668918,,"Stock(conId=265598, symbol='AAPL', right='0', ...",,...,,False,,228.690000,-1.540000,USD,1055.86,DU3180649,204.929105,1.115947
2024-09-19 22:15:36.758930000,SIE,STK,2.0,0.155796,169.530000,169.839996,0.182856,,"Stock(conId=14217, symbol='SIE', right='0', pr...",,...,,False,,339.680000,0.620000,EUR,0.00,DU3180649,339.680000,1.000000
2024-09-19 22:15:36.758945000,VOW,STK,2.0,0.090676,104.575000,98.849998,-5.474541,,"Stock(conId=14232, symbol='VOW', right='0', pr...",,...,,False,,197.700000,-11.450000,EUR,0.00,DU3180649,197.700000,1.000000
2024-09-19 22:32:27.552756000,AAPL,STK,1.0,0.000000,229.230000,229.230000,0.000000,TAAPL,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=2655981, symbol='AA...",...,2024-09-19,True,2024-09-19 23:10:17,0.000000,0.000000,USD,0.00,DU3180649,0.000000,1.115947
2024-09-19 22:45:57.517694001,AAPL,STK,1.0,0.000000,229.230000,228.770004,0.000000,TAAPL,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=2655981, symbol='AA...",...,2024-09-19,True,2024-09-19 23:10:17,0.000000,0.000000,USD,0.00,DU3180649,0.000000,1.115947
2024-09-19 23:02:19.483648000,AAPL,STK,2.0,0.000000,229.230000,229.230000,0.000000,TAAPL,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=2655981, symbol='AA...",...,2024-09-19,True,2024-09-19 23:10:17,0.000000,0.000000,USD,0.00,DU3180649,0.000000,1.115947
2024-09-19 23:05:21.680941001,AAPL,STK,2.0,0.000000,229.230000,228.600006,0.000000,TAAPL,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=2655981, symbol='AA...",...,2024-09-19,True,2024-09-19 23:10:17,0.000000,0.000000,USD,0.00,DU3180649,0.000000,1.115947
2024-09-19 23:05:21.680978000,SIE,STK,2.0,0.155795,169.530000,169.839996,0.182856,,"Stock(conId=14217, symbol='SIE', right='0', pr...",,...,,False,,339.679993,0.309996,EUR,0.00,DU3180649,339.679993,1.000000
2024-09-19 23:05:21.680983000,VOW,STK,2.0,0.090676,104.575000,98.849998,-5.474541,,"Stock(conId=14232, symbol='VOW', right='0', pr...",,...,,False,,197.699997,-5.725002,EUR,0.00,DU3180649,197.699997,1.000000
2024-09-19 23:10:31.343394000,AAPL,STK,1.0,0.093963,230.230052,228.619995,-0.699325,,"Stock(conId=265598, symbol='AAPL', right='0', ...",,...,,False,,228.619995,-1.610057,USD,1055.86,DU3180649,204.866374,1.115947


In [5]:
self.delete_portfolio_library()

####  Before Trade Start the program is always matching the ib positions with the arcticdb positions
So we do this here

In [9]:
def load_portfolio_from_adb(self):
    '''Function that loads latest saved portfolio from ArcticDB'''
    today = datetime.date.today()
    df = self.portfolio_library.read(f"{self.account_id}",date_range=(today - pd.Timedelta(days=10), None)).data
    if df.empty:
        df = self.portfolio_library.read(f"{self.account_id}",row_range=(-5000,9999)).data

    # Slicing for only active positions
    df = df[df['deleted'] != True].copy()

    # Create a column of our index, to recreate the index later after grouping
    df['timestamp'] = df.index
    
    # Group by symbol, strategy and asset class to find their last updated value
    latest_portfolio = df.sort_index().groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
    latest_portfolio.set_index('timestamp',drop=True, inplace=True)
    return latest_portfolio



In [10]:
def match_ib_positions_with_arcticdb(self):
    if self.account_id in self.portfolio_library.list_symbols():
        df_ac = self.load_portfolio_from_adb()
    else:
        df_ib = self.get_positions_from_ib()
        self.save_portfolio(df_ib)
        return df_ib

    df_ib = self.get_positions_from_ib()
    df_merged = pd.DataFrame()

    # Iterate through the positions obtained from Interactive Brokers
    for _, row in df_ib.iterrows():
        symbol = row['symbol']
        asset_class = row['asset class']

        strategy_entries_in_ac = df_ac[(df_ac['symbol'] == symbol) & (df_ac['asset class'] == asset_class)]
        
        if strategy_entries_in_ac.empty: # no database entry, add position
            # print(f"{asset_class}:{symbol} not in ArcticDB. Appending df_ib row to df_merged")
            df_merged = pd.concat([df_merged, pd.DataFrame([row])])
        else:
            strategy_entry_updated = self.update_and_aggregate_data(strategy_entries_in_ac, row)
            # print(f"{asset_class}:{symbol} in ArcticDB. Updated and aggregating data")
            df_merged = pd.concat([df_merged, strategy_entry_updated])
            
            if row['position'] - strategy_entry_updated.position.sum() != 0:
                # Handle the residual and concat to df_merged
                # print(f"{asset_class}:{symbol} IB position does not equal ArcticDB's Position")
                residual = self.handle_residual(strategy_entries_in_ac, row)
                df_merged = pd.concat([df_merged, residual])

    # Now, handle ArcticDB positions that aren't represented in the broker's data (e.g., strategies with net-zero positions)
    for _, row in df_ac.iterrows():
        symbol = row['symbol']
        asset_class = row['asset class']

        # Check if this position is not already accounted for in df_merged, then update market data
        if df_merged[(df_merged['symbol'] == symbol) & (df_merged['asset class'] == asset_class)].empty:
            # Update the market data for stale entries
            row = self.update_market_data_for_arcticdb_positions(row)

            # Otherwise, maintain the strategy-specific position, even if the broker doesn't report it
            df_merged = pd.concat([df_merged, pd.DataFrame([row])])

    self.save_portfolio(df_merged)
    self.save_account_pnl()
    
    return df_merged


In [16]:
if self.account_id in self.portfolio_library.list_symbols():
    df_ac = self.load_portfolio_from_adb()
else:
    df_ib = self.get_positions_from_ib()
    df_to_save = df_ib
    print("no arcticdb entry found, need to save df_ib (df_to_save)")
    #self.save_portfolio(df_ib)
    

no arcticdb entry found, need to save df_ib (df_to_save)


In [12]:
df_ib

,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-19 22:10:24.176793,AAPL,STK,1.0,0.094037,230.230052,228.800003,-0.621139,,"Stock(conId=265598, symbol='AAPL', right='0', ...",,...,,False,,228.80,-1.43,USD,1055.86,DU3180649,205.027676,1.115947
2024-09-19 22:10:24.176865,SIE,STK,2.0,0.155796,169.530000,169.839996,0.182856,,"Stock(conId=14217, symbol='SIE', right='0', pr...",,...,,False,,339.68,0.62,EUR,0.00,DU3180649,339.680000,1.000000
2024-09-19 22:10:24.176875,VOW,STK,2.0,0.090676,104.575000,98.849998,-5.474541,,"Stock(conId=14232, symbol='VOW', right='0', pr...",,...,,False,,197.70,-11.45,EUR,0.00,DU3180649,197.700000,1.000000


In [41]:
def save_portfolio(self, df_to_save):
    '''Function that saves all positions in ArcticDB in portfolio/"account_id".'''
    if df_to_save.empty:
        return
    df_to_save = normalize_columns(self,df_to_save)

    # Drop rows where the position is zero
    df_to_save = df_to_save[df_to_save['position'] != 0]

    try:       
        if self.account_id in self.portfolio_library.list_symbols():
            print(f"Updating arcticdb entry {self.account_id} in library 'portfolio'")
            self.portfolio_library.update(f'{self.account_id}', df_to_save, prune_previous_versions=True,upsert=True)
        else:
            print(f"Creating an arcticdb entry {self.account_id} in library 'portfolio'")
            self.portfolio_library.write(f'{self.account_id}',df_to_save, prune_previous_versions = True)#,  validate_index=True)
    except Exception as e:
        print(f"Error occured while saving: {e}")

def normalize_columns(self, df):
    df = df.copy()
    df['contract'] = df['contract'].astype(str)
    df['trade'] = df['trade'].astype(str)
    df['trade_context'] = df['trade_context'].astype(str)

    # Convert index to datetime
    df.index = pd.to_datetime(df.index, errors='coerce')

    # # Create the 'timestamp' column if it doesn't exist
    # if 'timestamp' not in df.columns:
    #     df['timestamp'] = df.index

    # # Convert the 'timestamp' column to datetime
    # df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

    # # Compare index and 'timestamp' column, and keep the newer value
    # df['timestamp'] = df.apply(lambda row: max(row.name, row['timestamp']) if pd.notnull(row['timestamp']) else row.name, axis=1)

    # # Ensure all rows have a valid timestamp
    # for idx, i in zip(df.index, range(0, len(df))):
    #     if pd.isna(df.at[idx, 'timestamp']):
    #         # If timestamp is NaT or missing, generate a unique timestamp
    #         df.at[idx, 'timestamp'] = pd.Timestamp.now() + pd.to_timedelta(i, unit='ns')

    # # Set the 'timestamp' column as the index
    # df.set_index('timestamp', inplace=True, drop=True)

    return df.sort_index()

Here, we are normalizing without creating a timestamp column and setting it as index

In [22]:
# Saving the trade to the portfolio library
df_to_save = normalize_columns(self,df_to_save)

# Drop rows where the position is zero
df_to_save = df_to_save[df_to_save['position'] != 0]

try:       
    if self.account_id in self.portfolio_library.list_symbols():
        print(f"Updating arcticdb entry {self.account_id} in library 'portfolio'")
        self.portfolio_library.update(f'{self.account_id}', df_to_save, prune_previous_versions=True,upsert=True)
    else:
        print(f"Creating an arcticdb entry {self.account_id} in library 'portfolio'")
        self.portfolio_library.write(f'{self.account_id}',df_to_save, prune_previous_versions = True)#,  validate_index=True)
except Exception as e:
    print(f"Error occured while saving: {e}")

Creating an arcticdb entry DU3180649 in library 'portfolio'


### Now, a portfolio is saved, and we can process a trade

### Create Trade Entry

In [36]:
trade_entry = create_trade_entry(self,"TAAPL",trade)
trade_entry

,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-19 22:32:27.552756,AAPL,STK,1.0,0.094214,229.23,229.23,0.0,TAAPL,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=265598, symbol='AAP...",...,,False,,229.23,0.0,USD,0.0,DU3180649,205.412999,1.115947


In [23]:
if detect_duplicate_trade(self,trade):
    print("duplicate trade detected")
else:
    print("no duplicate trade detected")

no duplicate trade detected


In [46]:
# Read the current portfolio data
if self.account_id in self.portfolio_library.list_symbols():
    df_ac_active = self.load_portfolio_from_adb()
else:
    df_ac_active = pd.DataFrame()

In [47]:
symbol = trade.contract.symbol
asset_class = trade.contract.secType

existing_position = df_ac_active[(df_ac_active['symbol'] == symbol) & (df_ac_active['asset class'] == asset_class) 
                                    & (df_ac_active['strategy'] == strategy_symbol)]
existing_position

,symbol,strategy,asset class,position,% of nav,averageCost,marketPrice,pnl %,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-19 22:32:27.552756,AAPL,TAAPL,STK,1.0,0.094214,229.23,229.23,0.0,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=265598, symbol='AAP...",...,,False,,229.23,0.0,USD,0.0,DU3180649,205.412999,1.115947


In [33]:
if existing_position.empty:
    # Simply append new position if it doesn't exist
    print(f"No existing position found for {symbol} in strategy '{strategy_symbol}'. Creating a new entry.")


No existing position found for AAPL in strategy 'TAAPL'. Creating a new entry.


In [42]:
save_portfolio(self,trade_entry)

Updating arcticdb entry DU3180649 in library 'portfolio'


### Now check the updated portfolio

In [48]:
load_portfolio_from_adb(self)

,symbol,strategy,asset class,position,% of nav,averageCost,marketPrice,pnl %,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-19 22:15:36.758804,AAPL,,STK,1.0,0.093992,230.230052,228.690002,-0.668918,"Stock(conId=265598, symbol='AAPL', right='0', ...",,...,,False,,228.69,-1.54,USD,1055.86,DU3180649,204.929105,1.115947
2024-09-19 22:32:27.552756,AAPL,TAAPL,STK,1.0,0.094214,229.230000,229.230000,0.000000,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=265598, symbol='AAP...",...,,False,,229.23,0.00,USD,0.00,DU3180649,205.412999,1.115947
2024-09-19 22:15:36.758930,SIE,,STK,2.0,0.155796,169.530000,169.839996,0.182856,"Stock(conId=14217, symbol='SIE', right='0', pr...",,...,,False,,339.68,0.62,EUR,0.00,DU3180649,339.680000,1.000000
2024-09-19 22:15:36.758945,VOW,,STK,2.0,0.090676,104.575000,98.849998,-5.474541,"Stock(conId=14232, symbol='VOW', right='0', pr...",,...,,False,,197.70,-11.45,EUR,0.00,DU3180649,197.700000,1.000000


In [49]:
self.match_ib_positions_with_arcticdb()

Updating arcticdb entry DU3180649 in library 'portfolio'
Equity value saved to 'pnl' library for account DU3180649


,symbol,strategy,asset class,position,% of nav,averageCost,marketPrice,pnl %,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
2024-09-19 22:45:57.517694000,AAPL,,STK,0.0,0.000000,inf,228.770004,-100.000000,"Stock(conId=265598, symbol='AAPL', right='0', ...",,...,,False,,0.000000,-inf,USD,1055.86,DU3180649,0.000000,1.115947
2024-09-19 22:45:57.517694001,AAPL,TAAPL,STK,1.0,0.094025,229.230,228.770004,-0.200670,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=265598, symbol='AAP...",...,,False,,228.770004,-0.459996,USD,0.00,DU3180649,205.000797,1.115947
2024-09-19 22:45:57.517722000,SIE,,STK,2.0,0.155796,169.530,169.839996,0.182856,"Stock(conId=14217, symbol='SIE', right='0', pr...",,...,,False,,339.679993,0.309996,EUR,0.00,DU3180649,339.679993,1.000000
2024-09-19 22:45:57.517735000,VOW,,STK,2.0,0.090676,104.575,98.849998,-5.474541,"Stock(conId=14232, symbol='VOW', right='0', pr...",,...,,False,,197.699997,-5.725002,EUR,0.00,DU3180649,197.699997,1.000000


In [50]:
self.portfolio_library.read(self.account_id).data

,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-19 22:15:36.758804000,AAPL,STK,1.0,0.093992,230.230052,228.690002,-0.668918,,"Stock(conId=265598, symbol='AAPL', right='0', ...",,...,,False,,228.690000,-1.540000,USD,1055.86,DU3180649,204.929105,1.115947
2024-09-19 22:15:36.758930000,SIE,STK,2.0,0.155796,169.530000,169.839996,0.182856,,"Stock(conId=14217, symbol='SIE', right='0', pr...",,...,,False,,339.680000,0.620000,EUR,0.00,DU3180649,339.680000,1.000000
2024-09-19 22:15:36.758945000,VOW,STK,2.0,0.090676,104.575000,98.849998,-5.474541,,"Stock(conId=14232, symbol='VOW', right='0', pr...",,...,,False,,197.700000,-11.450000,EUR,0.00,DU3180649,197.700000,1.000000
2024-09-19 22:32:27.552756000,AAPL,STK,1.0,0.094214,229.230000,229.230000,0.000000,TAAPL,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=265598, symbol='AAP...",...,,False,,229.230000,0.000000,USD,0.00,DU3180649,205.412999,1.115947
2024-09-19 22:45:57.517694001,AAPL,STK,1.0,0.094025,229.230000,228.770004,-0.200670,TAAPL,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=265598, symbol='AAP...",...,,False,,228.770004,-0.459996,USD,0.00,DU3180649,205.000797,1.115947
2024-09-19 22:45:57.517722000,SIE,STK,2.0,0.155796,169.530000,169.839996,0.182856,,"Stock(conId=14217, symbol='SIE', right='0', pr...",,...,,False,,339.679993,0.309996,EUR,0.00,DU3180649,339.679993,1.000000
2024-09-19 22:45:57.517735000,VOW,STK,2.0,0.090676,104.575000,98.849998,-5.474541,,"Stock(conId=14232, symbol='VOW', right='0', pr...",,...,,False,,197.699997,-5.725002,EUR,0.00,DU3180649,197.699997,1.000000


If everything works, we can now process a new trade for that symbol and see if the position is aggregated correctly

In [51]:
strategy_symbol = "TAAPL"
trade = Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=499, clientId=1, permId=1987284107, action='BUY', totalQuantity=1.0, lmtPrice=0.0, auxPrice=0.0, orderRef='TAAPL', algoStrategy='Adaptive', algoParams=[TagValue(tag='adaptivePriority', value='Urgent')]), orderStatus=OrderStatus(orderId=499, status='Filled', filled=1.0, remaining=0.0, avgFillPrice=229.23, permId=1987284107, parentId=0, lastFillPrice=229.23, clientId=1, whyHeld='', mktCapPrice=0.0), fills=[Fill(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), execution=Execution(execId='0000dc8f.66ebbc62.01.01', time=datetime.datetime(2024, 9, 19, 19, 14, 49, tzinfo=datetime.timezone.utc), acctNumber='DU3180649', exchange='IBKRATS', side='BOT', shares=1.0, price=229.23, permId=1987284107, clientId=1, orderId=499, liquidation=0, cumQty=1.0, avgPrice=229.23, orderRef='TAAPL', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2, pendingPriceRevision=False), commissionReport=CommissionReport(execId='0000dc8f.66ebbc62.01.01', commission=1.000052, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc))], log=[TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 48, 706996, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Submitted', message='Fill 1.0@229.23', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Filled', message='', errorCode=0)], advancedError='')

In [52]:
# Create a new trade entry
trade_entry = create_trade_entry(self,"TAAPL",trade)

if detect_duplicate_trade(self,trade):
    print("duplicate trade detected")
else:
    print("no duplicate trade detected")


Duplicate trade detected: Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=499, clientId=1, permId=1987284107, action='BUY', totalQuantity=1.0, lmtPrice=0.0, auxPrice=0.0, orderRef='TAAPL', algoStrategy='Adaptive', algoParams=[TagValue(tag='adaptivePriority', value='Urgent')]), orderStatus=OrderStatus(orderId=499, status='Filled', filled=1.0, remaining=0.0, avgFillPrice=229.23, permId=1987284107, parentId=0, lastFillPrice=229.23, clientId=1, whyHeld='', mktCapPrice=0.0), fills=[Fill(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), execution=Execution(execId='0000dc8f.66ebbc62.01.01', time=datetime.datetime(2024, 9, 19, 19, 14, 49, tzinfo=datetime.timezone.utc), acctNumber='DU3180649', exchange='IBKRATS', side='BOT', shares=1.0, price=229.23, permId=1987284107, clien

Duplicate trade detection works after position was saved to arcticdb for the first time

In [53]:
# Change Trade Object in conId to mimick new trade
trade = Trade(contract=Stock(conId=2655981, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=499, clientId=1, permId=1987284107, action='BUY', totalQuantity=1.0, lmtPrice=0.0, auxPrice=0.0, orderRef='TAAPL', algoStrategy='Adaptive', algoParams=[TagValue(tag='adaptivePriority', value='Urgent')]), orderStatus=OrderStatus(orderId=499, status='Filled', filled=1.0, remaining=0.0, avgFillPrice=229.23, permId=1987284107, parentId=0, lastFillPrice=229.23, clientId=1, whyHeld='', mktCapPrice=0.0), fills=[Fill(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), execution=Execution(execId='0000dc8f.66ebbc62.01.01', time=datetime.datetime(2024, 9, 19, 19, 14, 49, tzinfo=datetime.timezone.utc), acctNumber='DU3180649', exchange='IBKRATS', side='BOT', shares=1.0, price=229.23, permId=1987284107, clientId=1, orderId=499, liquidation=0, cumQty=1.0, avgPrice=229.23, orderRef='TAAPL', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2, pendingPriceRevision=False), commissionReport=CommissionReport(execId='0000dc8f.66ebbc62.01.01', commission=1.000052, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc))], log=[TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 48, 706996, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Submitted', message='Fill 1.0@229.23', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Filled', message='', errorCode=0)], advancedError='')

In [54]:
# Create a new trade entry
trade_entry = create_trade_entry(self,"TAAPL",trade)

if detect_duplicate_trade(self,trade):
    print("duplicate trade detected")
else:
    print("no duplicate trade detected")


no duplicate trade detected


In [56]:
# Read the current portfolio data
if self.account_id in self.portfolio_library.list_symbols():
    df_ac_active = self.load_portfolio_from_adb()
else:
    df_ac_active = pd.DataFrame()

symbol = trade.contract.symbol
asset_class = trade.contract.secType

existing_position = df_ac_active[(df_ac_active['symbol'] == symbol) & (df_ac_active['asset class'] == asset_class) 
                                    & (df_ac_active['strategy'] == strategy_symbol)]
existing_position

,symbol,strategy,asset class,position,% of nav,averageCost,marketPrice,pnl %,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-19 22:45:57.517694001,AAPL,TAAPL,STK,1.0,0.094025,229.23,228.770004,-0.20067,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=265598, symbol='AAP...",...,,False,,228.770004,-0.459996,USD,0.0,DU3180649,205.000797,1.115947


In [58]:
if existing_position.empty:
        # Simply append new position if it doesn't exist
        print(f"Processing a new trade: Saving {symbol} for strategy '{strategy_symbol}'")
        processed_trade_df = trade_entry
        
else:
    if len(existing_position) > 1:
        print(existing_position)
        print(f"Error: More than one entry of {asset_class}:{symbol} under '{strategy_symbol}'.")
        
    else:
        if existing_position.position.item() + trade_entry.position.item() == 0:
            print(f"Processing a new trade: Closing {symbol} in strategy '{strategy_symbol}'")
            self.close_position(existing_position, trade_entry)
            
        else:
            # Aggregating new trade to an existing position that's not a close
            print(f"Processing a new trade: Aggregating {symbol} to strategy '{strategy_symbol}'")
            processed_trade_df = self.aggregate_positions(existing_position, trade_entry)

Processing a new trade: Aggregating AAPL to strategy 'TAAPL'


In [61]:
processed_trade_df

,symbol,strategy,asset class,position,% of nav,averageCost,marketPrice,pnl %,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-19 23:02:19.483648,AAPL,TAAPL,STK,2.0,0.188427,229.23,229.23,-0.20067,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=2655981, symbol='AA...",...,,False,,458.46,0.0,USD,0.0,DU3180649,410.825999,1.115947


In [63]:
# If processed_trade_df looks good, we can save it
save_portfolio(self,processed_trade_df)

Updating arcticdb entry DU3180649 in library 'portfolio'


### Load Portfolio again and if it looks good, we can try to process a new trade and see if closing works

In [64]:
self.match_ib_positions_with_arcticdb()

Updating arcticdb entry DU3180649 in library 'portfolio'
Equity value saved to 'pnl' library for account DU3180649


,symbol,strategy,asset class,position,% of nav,averageCost,marketPrice,pnl %,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
2024-09-19 23:05:21.680941000,AAPL,,STK,-1.0,-0.093954,228.229948,228.600006,-0.162143,"Stock(conId=265598, symbol='AAPL', right='0', ...",,...,,False,,-228.600006,0.370058,USD,1055.86,DU3180649,-204.848462,1.115947
2024-09-19 23:05:21.680941001,AAPL,TAAPL,STK,2.0,0.187909,229.230000,228.600006,-0.274830,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=2655981, symbol='AA...",...,,False,,457.200012,-0.629994,USD,0.00,DU3180649,409.696924,1.115947
2024-09-19 23:05:21.680978000,SIE,,STK,2.0,0.155795,169.530000,169.839996,0.182856,"Stock(conId=14217, symbol='SIE', right='0', pr...",,...,,False,,339.679993,0.309996,EUR,0.00,DU3180649,339.679993,1.000000
2024-09-19 23:05:21.680983000,VOW,,STK,2.0,0.090676,104.575000,98.849998,-5.474541,"Stock(conId=14232, symbol='VOW', right='0', pr...",,...,,False,,197.699997,-5.725002,EUR,0.00,DU3180649,197.699997,1.000000


In [67]:
# Changing Trade Object in Action to SELL and qty = 2 to try to close
trade = Trade(contract=Stock(conId=2655981, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=499, clientId=1, permId=1987284107, action='SELL', totalQuantity=2.0, lmtPrice=0.0, auxPrice=0.0, orderRef='TAAPL', algoStrategy='Adaptive', algoParams=[TagValue(tag='adaptivePriority', value='Urgent')]), orderStatus=OrderStatus(orderId=499, status='Filled', filled=1.0, remaining=0.0, avgFillPrice=229.23, permId=1987284107, parentId=0, lastFillPrice=229.23, clientId=1, whyHeld='', mktCapPrice=0.0), fills=[Fill(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='AAPL', tradingClass='NMS'), execution=Execution(execId='0000dc8f.66ebbc62.01.01', time=datetime.datetime(2024, 9, 19, 19, 14, 49, tzinfo=datetime.timezone.utc), acctNumber='DU3180649', exchange='IBKRATS', side='BOT', shares=1.0, price=229.23, permId=1987284107, clientId=1, orderId=499, liquidation=0, cumQty=1.0, avgPrice=229.23, orderRef='TAAPL', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2, pendingPriceRevision=False), commissionReport=CommissionReport(execId='0000dc8f.66ebbc62.01.01', commission=1.000052, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc))], log=[TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 48, 706996, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Submitted', message='Fill 1.0@229.23', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 9, 19, 19, 14, 49, 712313, tzinfo=datetime.timezone.utc), status='Filled', message='', errorCode=0)], advancedError='')

In [68]:
# Create a new trade entry
trade_entry = create_trade_entry(self,"TAAPL",trade)

if detect_duplicate_trade(self,trade):
    print("duplicate trade detected")
else:
    print("no duplicate trade detected")


no duplicate trade detected


In [69]:
trade_entry

,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-19 23:09:05.934886,AAPL,STK,-2.0,-0.188427,229.23,229.23,0.0,TAAPL,"Stock(conId=2655981, symbol='AAPL', exchange='...","Trade(contract=Stock(conId=2655981, symbol='AA...",...,,False,,-458.46,0.0,USD,0.0,DU3180649,-410.825999,1.115947


In [70]:
# Read the current portfolio data
if self.account_id in self.portfolio_library.list_symbols():
    df_ac_active = self.load_portfolio_from_adb()
else:
    df_ac_active = pd.DataFrame()

symbol = trade.contract.symbol
asset_class = trade.contract.secType

existing_position = df_ac_active[(df_ac_active['symbol'] == symbol) & (df_ac_active['asset class'] == asset_class) 
                                    & (df_ac_active['strategy'] == strategy_symbol)]
existing_position

,symbol,strategy,asset class,position,% of nav,averageCost,marketPrice,pnl %,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-19 23:05:21.680941001,AAPL,TAAPL,STK,2.0,0.187909,229.23,228.600006,-0.27483,"Stock(conId=265598, symbol='AAPL', exchange='S...","Trade(contract=Stock(conId=2655981, symbol='AA...",...,,False,,457.200012,-0.629994,USD,0.0,DU3180649,409.696924,1.115947


In [71]:
if existing_position.empty:
        # Simply append new position if it doesn't exist
        print(f"Processing a new trade: Saving {symbol} for strategy '{strategy_symbol}'")
        processed_trade_df = trade_entry
        
else:
    if len(existing_position) > 1:
        print(existing_position)
        print(f"Error: More than one entry of {asset_class}:{symbol} under '{strategy_symbol}'.")
        
    else:
        if existing_position.position.item() + trade_entry.position.item() == 0:
            print(f"Processing a new trade: Closing {symbol} in strategy '{strategy_symbol}'")
            self.close_position(existing_position, trade_entry)
            
        else:
            # Aggregating new trade to an existing position that's not a close
            print(f"Processing a new trade: Aggregating {symbol} to strategy '{strategy_symbol}'")
            processed_trade_df = self.aggregate_positions(existing_position, trade_entry)

Processing a new trade: Closing AAPL in strategy 'TAAPL'
Updating arcticdb entry DU3180649 in library 'portfolio'
Closed position for AAPL STK with strategy TAAPL


In [72]:
self.match_ib_positions_with_arcticdb()

Updating arcticdb entry DU3180649 in library 'portfolio'
Equity value saved to 'pnl' library for account DU3180649


,symbol,strategy,asset class,position,% of nav,averageCost,marketPrice,pnl %,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
2024-09-19 23:10:31.343394,AAPL,,STK,1.0,0.093963,230.230052,228.619995,-0.699325,"Stock(conId=265598, symbol='AAPL', right='0', ...",,...,,False,,228.619995,-1.610057,USD,1055.86,DU3180649,204.866374,1.115947
2024-09-19 23:10:31.343435,SIE,,STK,2.0,0.155796,169.530000,169.839996,0.182856,"Stock(conId=14217, symbol='SIE', right='0', pr...",,...,,False,,339.679993,0.309996,EUR,0.00,DU3180649,339.679993,1.000000
2024-09-19 23:10:31.343443,VOW,,STK,2.0,0.090676,104.575000,98.849998,-5.474541,"Stock(conId=14232, symbol='VOW', right='0', pr...",,...,,False,,197.699997,-5.725002,EUR,0.00,DU3180649,197.699997,1.000000


In [74]:
self.portfolio_library.read(self.account_id).data.columns

Index(['symbol', 'asset class', 'position', '% of nav', 'averageCost',
       'marketPrice', 'pnl %', 'strategy', 'contract', 'trade',
       'trade_context', 'open_dt', 'close_dt', 'deleted', 'delete_dt',
       'marketValue', 'unrealizedPNL', 'currency', 'realizedPNL', 'account',
       'marketValue_base', 'fx_rate'],
      dtype='object')

In [ ]:
def process_new_trade(self, strategy_symbol, trade):
    '''Function that processes an ib_insync trade object and stores it in the ArcticDB'''
    # Create a Dataframe compatible with our ArcticDB data structure
    print(trade)
    trade_df = create_trade_entry(self,strategy_symbol, trade)
    print(trade_df)
    # Check for duplicate trades and exit function if True
    if detect_duplicate_trade(self,trade):
        return

    symbol = trade.contract.symbol
    asset_class = trade.contract.secType

    # Read the current portfolio data
    if self.account_id in self.portfolio_library.list_symbols():
        df_ac_active = self.load_portfolio_from_adb()
    else:
        df_ac_active = pd.DataFrame()

    existing_position = df_ac_active[(df_ac_active['symbol'] == symbol) & (df_ac_active['asset class'] == asset_class) 
                                    & (df_ac_active['strategy'] == strategy_symbol)]

    if existing_position.empty:
        # Simply append new position if it doesn't exist
        print(f"Processing a new trade: Saving {symbol} for strategy '{strategy_symbol}'")
        processed_trade_df = trade_df
        
    else:
        if len(existing_position) > 1:
            print(existing_position)
            print(f"Error: More than one entry of {asset_class}:{symbol} under '{strategy_symbol}'.")
            return
        else:
            if existing_position.position.item() + trade_df.position.item() == 0:
                print(f"Processing a new trade: Closing {symbol} in strategy '{strategy_symbol}'")
                self.close_position(existing_position, trade_df)
                return
            else:
                # Aggregating new trade to an existing position that's not a close
                print(f"Processing a new trade: Aggregating {symbol} to strategy '{strategy_symbol}'")
                processed_trade_df = self.aggregate_positions(existing_position, trade_df)
        
    # Save the updated positions
    self.save_portfolio(processed_trade_df)

In [7]:
self.get_positions_from_ib()

,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-19 22:03:05.550834,AAPL,STK,1.0,0.094094,230.230052,228.940002,-0.560331,,"Stock(conId=265598, symbol='AAPL', right='0', ...",,...,,False,,228.94,-1.29,USD,1055.86,DU3180649,205.15313,1.115947
2024-09-19 22:03:08.212426,SIE,STK,2.0,0.155794,169.530000,169.839996,0.182856,,"Stock(conId=14217, symbol='SIE', right='0', pr...",,...,,False,,339.68,0.62,EUR,0.00,DU3180649,339.68000,1.000000
2024-09-19 22:03:08.212454,VOW,STK,2.0,0.090675,104.575000,98.849998,-5.474541,,"Stock(conId=14232, symbol='VOW', right='0', pr...",,...,,False,,197.70,-11.45,EUR,0.00,DU3180649,197.70000,1.000000


In [8]:
df = self.match_ib_positions_with_arcticdb()
df

Updating arcticdb entry DU3180649 in library 'portfolio'
Equity value saved to 'pnl' library for account DU3180649


,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
2024-09-19 21:33:03.232488,AAPL,STK,1.0,0.094181,230.230052,229.100006,-0.490833,,"Stock(conId=265598, symbol='AAPL', right='0', ...",,...,,False,,229.100000,-1.130000,USD,1055.86,DU3180649,205.342324,1.115698
2024-09-19 21:33:03.802420,SIE,STK,2.0,0.155703,169.530000,169.740005,0.123875,,"Stock(conId=14217, symbol='SIE', right='0', pr...",,...,,False,,339.480011,0.210005,EUR,0.00,DU3180649,339.480011,1.000000
2024-09-19 21:33:03.802486,VOW,STK,2.0,0.090951,104.575000,99.150002,-5.187663,,"Stock(conId=14232, symbol='VOW', right='0', pr...",,...,,False,,198.300003,-5.424998,EUR,0.00,DU3180649,198.300003,1.000000


Arctic(config=S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb))